In [1]:
#import vectorbt as vbt
import numpy as np
import pandas as pd
import datetime
import plotly.express as px
from xbbg import blp
import os
import quantstats as qs
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Import custom modules with an alias
import bloomberg_data as bd
import transformations as tr

In [4]:
# Define the path to the CSV file
file_path = './Raw Data/spx_trender.csv'

# Load the CSV file into a DataFrame
data = pd.read_csv(file_path)

# Convert the 'Date' column to datetime format
data['Date'] = pd.to_datetime(data['Date'])

# Set the 'Date' column as the index
data.set_index('Date', inplace=True)

print(data.head())
print('----------------------------------------------------------------')
print('----------------------------------------------------------------')
print(data.tail())
print('----------------------------------------------------------------')
print('----------------------------------------------------------------')
print(data.info())

             Open   High    Low  Close  TrndrUp  TrndrDn
Date                                                    
1927-12-30  17.66  17.66  17.66  17.66      NaN      NaN
1928-01-03  17.76  17.76  17.76  17.76      NaN      NaN
1928-01-04  17.72  17.72  17.72  17.72      NaN      NaN
1928-01-05  17.55  17.55  17.55  17.55      NaN      NaN
1928-01-06  17.66  17.66  17.66  17.66      NaN      NaN
----------------------------------------------------------------
----------------------------------------------------------------
               Open     High      Low    Close  TrndrUp  TrndrDn
Date                                                            
2024-06-28  5488.48  5523.64  5451.12  5460.48  5299.87      NaN
2024-07-01  5471.08  5479.55  5446.53  5475.09  5299.87      NaN
2024-07-02  5461.84  5509.69  5458.43  5509.01  5299.87      NaN
2024-07-03  5507.44  5539.27  5507.42  5537.02  5302.24      NaN
2024-07-05  5537.91  5565.05  5531.63  5560.74  5344.18      NaN
----------------

In [18]:
# Create 'Up' column
data['Up'] = data['TrndrUp'].fillna(method='ffill')

# Create 'Down' column
data['Down'] = data['TrndrDn'].fillna(method='ffill')

# Drop rows where 'Up' or 'Down' columns are still NaN
data.dropna(subset=['Up', 'Down'], inplace=True)

# Create '% vs Up' column
data['% vs Up'] = (data['Close'] / data['Up']) - 1

# Create '% vs Down' column
data['% vs Down'] = (data['Close'] / data['Down']) - 1

# Create '% vs Up' column
data['% vs Up'] = (data['Close'] / data['Up']) - 1

# Create '% vs Down' column
data['% vs Down'] = (data['Close'] / data['Down']) - 1

# Define conditions
conditions = [
    (data['TrndrUp'].notna() & (data['Up'] > data['Down'])),
    (data['TrndrUp'].notna() & (data['Up'] < data['Down'])),
    (data['TrndrDn'].notna() & (data['Up'] < data['Down'])),
    (data['TrndrDn'].notna() & (data['Up'] > data['Down']))
]

# Define corresponding values
values = [2, 1, -1, -2]

# Create 'State of Market' column
data['State of Market'] = np.select(conditions, values)

# Create '% vs Trend' column based on 'State of Market'
data['% vs Trend'] = np.where(data['State of Market'].isin([2, -1]), data['% vs Up'], data['% vs Down'])


In [19]:
data.tail(60)

,Open,High,Low,Close,TrndrUp,TrndrDn,Up,Down,% vs Up,% vs Down,State of Market,% vs Trend
Date,,,,,,,,,,,,
2024-04-10,5167.88,5178.43,5138.70,5160.64,5073.21,NaN,5073.21,4293.80,0.017234,0.201882,2,0.017234
2024-04-11,5172.95,5211.78,5138.77,5199.06,5073.21,NaN,5073.21,4293.80,0.024807,0.210830,2,0.024807
2024-04-12,5171.51,5175.03,5107.94,5123.41,5073.21,NaN,5073.21,4293.80,0.009895,0.193211,2,0.009895
2024-04-15,5149.67,5168.43,5052.47,5061.82,NaN,5485.20,5073.21,5485.20,-0.002245,-0.077186,-1,-0.002245
2024-04-16,5064.59,5079.84,5039.83,5051.41,NaN,5435.87,5073.21,5435.87,-0.004297,-0.070726,-1,-0.004297
2024-04-17,5068.97,5077.96,5007.25,5022.21,NaN,5401.13,5073.21,5401.13,-0.010053,-0.070156,-1,-0.010053
2024-04-18,5031.52,5056.66,5001.89,5011.12,NaN,5376.50,5073.21,5376.50,-0.012239,-0.067959,-1,-0.012239
2024-04-19,5005.44,5019.02,4953.56,4967.23,NaN,5321.74,5073.21,5321.74,-0.020890,-0.066615,-1,-0.020890
2024-04-22,4987.33,5038.84,4969.40,5010.60,NaN,5298.11,5073.21,5298.11,-0.012341,-0.054267,-1,-0.012341


In [21]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Assuming 'data' DataFrame is already loaded and 'Date' is set as index

# Create 'Up' column
data['Up'] = data['TrndrUp'].fillna(method='ffill')

# Create 'Down' column
data['Down'] = data['TrndrDn'].fillna(method='ffill')

# Drop rows where 'Up' or 'Down' columns are still NaN
data.dropna(subset=['Up', 'Down'], inplace=True)

# Create '% vs Up' column
data['% vs Up'] = (data['Close'] / data['Up']) - 1

# Create '% vs Down' column
data['% vs Down'] = (data['Close'] / data['Down']) - 1

# Define conditions
conditions = [
    (data['TrndrUp'].notna() & (data['Up'] > data['Down'])),
    (data['TrndrUp'].notna() & (data['Up'] < data['Down'])),
    (data['TrndrDn'].notna() & (data['Up'] < data['Down'])),
    (data['TrndrDn'].notna() & (data['Up'] > data['Down']))
]

# Define corresponding values
values = [2, 1, -1, -2]

# Create 'State of Market' column
data['State of Market'] = np.select(conditions, values)

# Create '% vs Trend' column based on 'State of Market'
data['% vs Trend'] = np.where(data['State of Market'].isin([2, -1]), data['% vs Up'], data['% vs Down'])

# Filter the data for each state of market
data_state_2 = data[data['State of Market'] == 2]
data_state_1 = data[data['State of Market'] == 1]
data_state_neg1 = data[data['State of Market'] == -1]
data_state_neg2 = data[data['State of Market'] == -2]

# Create subplots
fig = make_subplots(rows=4, cols=1, subplot_titles=("State of Market = 2", "State of Market = 1", "State of Market = -1", "State of Market = -2"))

# Add traces for each state
fig.add_trace(go.Scatter(x=data_state_2.index, y=data_state_2['% vs Trend'], mode='lines', name='% vs Trend (State 2)'), row=1, col=1)
fig.add_trace(go.Scatter(x=data_state_1.index, y=data_state_1['% vs Trend'], mode='lines', name='% vs Trend (State 1)'), row=2, col=1)
fig.add_trace(go.Scatter(x=data_state_neg1.index, y=data_state_neg1['% vs Trend'], mode='lines', name='% vs Trend (State -1)'), row=3, col=1)
fig.add_trace(go.Scatter(x=data_state_neg2.index, y=data_state_neg2['% vs Trend'], mode='lines', name='% vs Trend (State -2)'), row=4, col=1)

# Update layout
fig.update_layout(height=800, width=1000, title_text="% vs Trend for Each State of Market", showlegend=False)

# Show the plot
fig.show()



In [24]:
import vectorbt as vbt
import pandas as pd
import numpy as np

# Assuming 'data' DataFrame is already prepared

# Define the entry and exit signals based on '% vs Trend > 0'
entries = data['% vs Trend'] > 0
exits = data['% vs Trend'] <= 0

# Create a vectorbt portfolio
portfolio = vbt.Portfolio.from_signals(
    close=data['Close'],
    entries=entries,
    exits=exits,
    init_cash=100_000,  # Example initial cash amount
    fees=0.001  # Example fee rate (0.1%)
)

# Run the backtest
portfolio_stats = portfolio.stats()

# Print the statistics of the backtest
print(portfolio_stats)



Start                         1929-02-08 00:00:00
End                           2024-07-05 00:00:00
Period                                      23966
Start Value                              100000.0
End Value                         14193617.631978
Total Return [%]                     14093.617632
Benchmark Return [%]                 22652.618658
Max Gross Exposure [%]                      100.0
Total Fees Paid                    5057933.215931
Max Drawdown [%]                        48.254841
Max Drawdown Duration                      4450.0
Total Trades                                  718
Total Closed Trades                           717
Total Open Trades                               1
Open Trade PnL                       674129.22261
Win Rate [%]                            28.312413
Best Trade [%]                          48.833822
Worst Trade [%]                         -6.977602
Avg Winning Trade [%]                    6.284575
Avg Losing Trade [%]                    -1.340781
